In [1]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
np.random.seed(42)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'D2Coding'
plt.rcParams['axes.unicode_minus'] = False

8. （3장에서 소개한） MNIST 데이터를 불러들여 훈련 세트, 검증 세트, 테스트 세트로 나눕
니다（예를 들면 훈련에 50,000개 샘플, 검증에 10,000개 샘플, 테스트에 10,000개 샘
플） . 그런 다음 랜덤 포레스트 분류기 , 엑스트라 트리 분류기, SVM 분류기 같은 여러 종
류의 분류기를 훈련시킵니다. 그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내
도록 이들을 간접 또는 직접 투표 방법을 사용해 앙상블로 연결해보세요. 앙상블을 얻고
나면 테스트 세트로 확인해보세요. 개개의 분류기와 비교해서 성능이 얼마나 향상되나요?

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val,test_size=10000,random_state=42)


랜덤포레스트,엑스트라트리 분류기, SVM, MLP분류 등 여러종류의 분류기를 훈련시킨다

In [5]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [6]:
from random import random


random_forest_clf=RandomForestClassifier(n_estimators=10,random_state=42)
extra_tree_clf=ExtraTreesClassifier(n_estimators=10,random_state=42)
svm_clf=LinearSVC(max_iter=10000,random_state=42)
mlp_clf=MLPClassifier(random_state=42)

In [7]:
estimators = [random_forest_clf,extra_tree_clf,svm_clf,mlp_clf]
for estimator in estimators:
    print("훈련 예측기: ",estimator)
    estimator.fit(X_train,y_train)

훈련 예측기:  RandomForestClassifier(n_estimators=10, random_state=42)
훈련 예측기:  ExtraTreesClassifier(n_estimators=10, random_state=42)
훈련 예측기:  LinearSVC(max_iter=10000, random_state=42)


c:\Users\nmh90\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


훈련 예측기:  MLPClassifier(random_state=42)


In [8]:
[estimator.score(X_val,y_val) for estimator in estimators]

[0.9469, 0.9492, 0.8558, 0.9626]

투표기반 분류기를 사용하여 앙상블로 연결

In [9]:
from sklearn.ensemble import VotingClassifier

In [10]:
named_estimators = [
    ("random_forest_clf",random_forest_clf),
    ("extra_trees_clf",extra_tree_clf),
    ("svm_clf",svm_clf),
    ("mlp_clf",mlp_clf),
]

In [11]:
voting_clf = VotingClassifier(named_estimators)

In [12]:
voting_clf.fit(X_train,y_train)

c:\Users\nmh90\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=10000, random_state=42)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [13]:
voting_clf.score(X_val,y_val)

0.9612

In [14]:
[estimator.score(X_val,y_val) for estimator in voting_clf.estimators_]

[0.0, 0.0, 0.0, 0.0]

In [15]:
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))])

예측기 목록 업데이트됨

In [16]:
voting_clf.estimators

[('random_forest_clf',
  RandomForestClassifier(n_estimators=10, random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(n_estimators=10, random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

훈련된 예측기 목록은 업데이트가 되지 않음

In [17]:
voting_clf.estimators_

[RandomForestClassifier(n_estimators=10, random_state=42),
 ExtraTreesClassifier(n_estimators=10, random_state=42),
 LinearSVC(max_iter=10000, random_state=42),
 MLPClassifier(random_state=42)]

훈련된 예측기에서 SVM 모델 제거

In [18]:
del voting_clf.estimators_[2]

In [19]:
voting_clf.score(X_val,y_val)

0.9639

0.9612 => 0.9639로 성능이 좋아졌다

Hard Voting
 각각의 모델들이 각각의 결과를 예측하면 단순하게 가장 많은 표를 얻은 결과를 선택하는것

Soft Voting
class별로 모델들이 예측한 probability를 합산해서 가장 높은 class를 선택

In [20]:
voting_clf.voting="soft"

In [21]:
voting_clf.score(X_val,y_val)

0.9695

soft voting을 하니 향상된 결과를 얻었다.

In [22]:
voting_clf.score(X_test,y_test)

0.9681

In [23]:
[estimator.score(X_test,y_test) for estimator in voting_clf.estimators_]

[0.0, 0.0, 0.0]

In [24]:
voting_clf.set_params(svm_clf=None)


VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(n_estimators=10,
                                                   random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))],
                 voting='soft')

9. 이전 연습문제의 각 분류기를 실행해서 검증 세트에서 예측을 만들고 그 결과로 새로운 훈
련 세트를 만들어보세요. 각 훈련 샘플은 하나의 이미지에 대한 전체 분류기의 예측을 담
은 벡터이고 타깃은 이미지의 클래스입니다. 새로운 훈련 세트에 분류기 하나를 훈련시켜
보세요. 축하합니다. 방금 블렌더를 훈련시켰습니다. 그리고 이 분류기를 모아서 스태킹
앙상블을 구성했습니다! 이제 테스트 세트에 앙상블을 평가해보세요. 테스트 세트의 각 이
미지에 대해 모든 분류기로 예측을 만들고 앙상블의 예측 결과를 만들기 위해 블렌더에 그
예측을 주입합니다. 앞서 만든 투표 분류기와 비교하면 어떤가요?


In [25]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [26]:
X_val_predictions


array([[5., 5., 5., 5.],
       [8., 8., 8., 8.],
       [2., 2., 2., 2.],
       ...,
       [7., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [27]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [28]:
rnd_forest_blender.oob_score_


0.9606

In [29]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [30]:
y_pred = rnd_forest_blender.predict(X_test_predictions)


In [31]:
from sklearn.metrics import accuracy_score


In [32]:
accuracy_score(y_test, y_pred)


0.9587

개개의 분류기보다는 좋은 결과를 가져온다